In [2]:
import argparse
import numpy as np
import pdb 
import torch, torchvision
from torch.utils.data import DataLoader
from torch.autograd import Variable

def update(x,y,t,s,A):
  for i in range(x-s,x+s+1):
     for j in range(y-s,y+s+1):
        if not(i==x and j==y) and i>0 and j>0 and i<34 and j<34:
	        A[i][j] = t
    

#Select the device to run the code 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ('Device: ' + str(device))

def calc_gradients(test_dataloader,model,max_iter,
                learning_rate,samples,filter_on,s,T,
		        targets=None,weight_loss2=1,batch_size=1,seq_len=40,):
	
	#Define the modifier and the optimizer

  #modif = torch.Tensor(1, seq_len, 2, 34, 34).fill_(1).to(device)
  #modifier = torch.nn.Parameter(modif, requires_grad=True)
  #optimizer = torch.optim.Adam([modifier], lr=learning_rate)
  min_loss = 1e-5
  prev_loss = 1e-5
  correct = np.zeros(max_iter)	
  for batch_index, (input_image, input_target, input_label) in enumerate(test_dataloader):

    modif = torch.Tensor(1, seq_len, 2, 34, 34).fill_(1).to(device)
    modifier = torch.nn.Parameter(modif, requires_grad=True)
    optimizer = torch.optim.Adam([modifier], lr=learning_rate)


    #if batch_index>samples:
     # continue
    model.eval()
    with torch.no_grad():	
      input_image, input_label = input_image.to(device), input_label.to(device)

    #Clean video prediction 
    print(f'Batch Number: {batch_index}/{len(test_dataloader)}')
    print('------------------prediction for clean video-------------------')
    input_image = Variable(input_image, requires_grad=True)
    output = model.forward(torch.reshape(input_image, (1,2312,1,1,350)))
    pre_label= snn.predict.getClass(output)
    p = torch.nn.Softmax(dim=1)		
    #numSpikes = torch.sum(output, 4, keepdim=True).cpu()	
    #probs= p(numSpikes.reshape((numSpikes.shape[0], -1)))
    print (f'Prediction: {pre_label}, Original_label: {input_label.cpu().numpy()}')

    print('------------------prediction for adversarial video-------------------')

    min_in = input_image.min().detach() #0
    max_in = input_image.max().detach() #1
    
    all_loss = []
    
    for iiter in range(max_iter):
      with open ('/content/modifier.txt','w') as file:
        file.write(str(modifier))      
      print(modifier[0,0,0,0,0])			
      input_image = Variable(input_image, requires_grad=True)
      #model.lstm.reset_hidden_state()
      
      #Frames to be perturbed
      indicator = [0]*350
      for i,x in enumerate(indicator):
        if (i>95 and i<125) or (i>175 and i<210) or (i>290 and i<310 ):
          x=1
    
      #Perturbating the frames
      #print(modifier.shape, input_image.shape)
      input_image=torch.reshape(input_image, (1,350,2,34,34))
      #print("input image shape:{}".format(input_image.shape))
      true_image = torch.clamp((modifier[0,0,:,:,:]+input_image[0,0,:,:,:]), min_in, max_in)
      true_image = torch.unsqueeze(true_image, 0)
      
      for ll in range(seq_len-1): #seq_len =1450
        if indicator[ll+1] == 1:
          mask_temp = torch.clamp((modifier[0,ll+1,:,:,:]+input_image[0,ll+1,:,:,:]), min_in, max_in)
        else:
          mask_temp = input_image[0,ll+1,:,:,:]
        mask_temp = torch.unsqueeze(mask_temp,0)
        true_image = torch.cat((true_image, mask_temp),0)
      #true_image = torch.unsqueeze(true_image, 0)

      #######		 Filter section 	######
      if filter_on :
        img = torch.reshape(true_image, (2,34,34,350)).cpu().detach()
        TD = snn.io.spikeArrayToEvent(img.numpy(), samplingTime=1)
        snn.io.encodeNpSpikes("/content/drive/My Drive/temp/img.npy", TD)
        data= np.load('/content/drive/My Drive/temp/img.npy'.format(i))
        data= data[data[:,3].argsort()] #sort elements by timestamp
        temp=np.zeros((34,34))
        real=[]
        for d in data:
          update(int(d[0]),int(d[1]),d[3],s,temp)
          if d[3]-temp[int(d[0])][int(d[1])]<T:
            real.append(d)
        real=np.stack(real, axis=0 )
        with open("/content/drive/My Drive/temp/fil.npy", "wb") as f:
          np.save(f,real)
        true_image = snn.io.readNpSpikes(
            "/content/drive/My Drive/temp/fil.npy"
            ).toSpikeTensor(torch.zeros((2,34,34,350)),
            samplingTime=1.0)
        true_image =  true_image.to(device)
      true_image = torch.reshape(true_image, (1,350,2,34,34))			
      #######		 End Filter Section 	 #######							
     
      #Prediction on the adversarial video
      output = model.forward(torch.reshape(true_image, (1,2312,1,1,350)))
      pre_label = snn.predict.getClass(output)
      numSpikes = torch.sum(output, 4, keepdim=True)
      numSpikes= torch.div(numSpikes,torch.max(numSpikes))	
      probs= p(numSpikes.reshape((numSpikes.shape[0], -1)))	 
      #extracting the probability of true label 
      zero_array = torch.zeros(10).to(device) ## 1450 invece di 11 e probs=p(output)
      zero_array[input_label.cpu()] = 1
      true_label_onehot = probs*zero_array
      true_label_prob = torch.sum(true_label_onehot, 1)
      #print (f'Prediction: {pre_label}, Original_label: {input_label.cpu().numpy()}')
      if pre_label.numpy()==input_label.cpu().numpy() : correct[iiter]+=1
      #print("""True label prob : {}\n zero_array[input_label.cpu()]: {} \n input_label.cpu(): {}""".format(true_label_prob,zero_array[input_label.cpu()],input_label.cpu() ))		
      #Loss
      if targets is None:
        loss1 = -torch.log(1 - true_label_prob + 1e-6)
      else:
        loss1 = -torch.log(true_label_prob + 1e-6)
      loss1 = torch.mean(loss1)
      input_image=torch.reshape(input_image, (1,350,2,34,34))
      loss2 = torch.sum(torch.sqrt(torch.mean(torch.pow((true_image-input_image+1e-6), 2), dim=0).mean(dim=2).mean(dim=2).mean(dim=1)))

      #norm_frame = torch.mean(torch.abs(modifier), dim=3).mean(dim=3).mean(dim=2) 

      loss = loss1 + weight_loss2 * loss2

      optimizer.zero_grad()
      loss.backward() # check retain_graph loss.backward(retain_graph=True)
      optimizer.step()

      if True: #iiter % max_iter == 0: 
        print (f'Probability for ground truth label : {true_label_prob.detach().cpu().numpy()}')
        if prev_loss < loss : 
          print(f'Iteration: [{iiter}/{max_iter}], Loss: {loss}(\u25b2), Loss1: {loss1}, Loss2: {loss2}')
        elif prev_loss > loss: 
          print(f'Iteration: [{iiter}/{max_iter}], Loss: {loss}(\u25bc), Loss1: {loss1}, Loss2: {loss2}')
        else: 
          print(f'Iteration: [{iiter}/{max_iter}], Loss: {loss}, Loss1: {loss1}, Loss2: {loss2}')
      prev_loss = loss

      break_condition = False
      if loss < min_loss:
        if torch.abs(loss-min_loss) < 0.0001:
            break_condition = True
            print ('Aborting early!')
        min_loss = loss

      # Empty cache
      if torch.cuda.is_available():
        torch.cuda.empty_cache()
      #print(numSpikes,totSpikes, totSpikes.shape, numSpikes.shape)
    # Save adversarial dataset as spike events
    image = torch.reshape(true_image, (2,34,34,350)).cpu().detach()
    TD = snn.io.spikeArrayToEvent(image.numpy(), samplingTime=1)
    snn.io.encodeNpSpikes("/content/prova/{}.npy".format(batch_index), TD)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()	
  print(correct)


def main(net):

  args = {
      'description': 'Sparse Adversarial Perturbations',
      'input_dir' : '/content/drive/My Drive/Test/',
      'num_iter': 20,
      'learning_rate': 100,
      'samples':11,
			'filter_on': False,
			's': 2,
			'T': 5,
      'target': None,
      'weight_loss2': 0,
      'split_path': None,
      'split_number': 1,
      'img_dim': 34,
      'channels': 2,
      'checkpoint_model': '../../../drive/My Drive/best/check.pt',
			}
  seq_len = 350 #number of frames in a video
  batch_size = 1
  targets = None
  image_shape = (args['channels'], args['img_dim'], args['img_dim'])
  net = net.to(device)
  net.train()
  print('Model Loaded Successfully!')
	#Call the function to generate the adversarial videos
  calc_gradients(
	testLoader,
	net,
	args['num_iter'],
	args['learning_rate'],
  args['samples'],
	args['filter_on'],
	args['s'],
	args['T'],
	targets,
	args['weight_loss2'],
	batch_size,
	seq_len,
	)

if __name__ == '__main__':
	main(net)

Device: cuda


NameError: name 'net' is not defined